In [1]:
import os
import fastavro
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from correction.compute import *

In [38]:
from alerce.api import AlerceAPI
client = AlerceAPI()

In [2]:
AVRO_PATH = "../data_examples/"

# Functions to manipulate example data

In [26]:
def read_avro(avro_path):
    with open(avro_path, "rb") as file:
        try:
            reader = fastavro.reader(file)
            data = reader.next()
        except Exception as e:
           return None
    return data

In [27]:
def get_avros(oid):
    path = os.path.join(AVRO_PATH, oid)
    avros = sorted(os.listdir(path))
    data = []
    for a in avros:
        content = read_avro(os.path.join(path, a))
        if content:
            data.append(content)
    return data

In [47]:
detection_columns = ['candid', 'jd', 'fid', 'pid', 'diffmaglim', 'isdiffpos', 'nid', 'ra', 'dec', 'magpsf', 'sigmapsf', 'magap', 'sigmagap', 'distnr', 'rb', 'rbversion', 'drb', 'drbversion', 'magapbig', 'sigmagapbig', 'rfid', 'magpsf_corr', 'sigmapsf_corr', 'sigmapsf_corr_ref', 'has_stamp', 'corr', 'flag', 'candid_alert', 'step_id_corr']
detection_table = pd.DataFrame(columns=detection_columns)

,candid,jd,fid,pid,diffmaglim,isdiffpos,nid,ra,dec,magpsf,...,sigmagapbig,rfid,magpsf_corr,sigmapsf_corr,sigmapsf_corr_ref,has_stamp,corr,flag,candid_alert,step_id_corr


In [28]:
oids = os.listdir(AVRO_PATH)

In [35]:
oid = oids[2]
example = get_avros(oid)
is_first_detection = {"object": True, 1: True, 2: True}
first_magnr = {}
df = pd.DataFrame()
colors = {1: 'g', 2: 'r'}

In [40]:
for idx, data in enumerate(example):
    # filter
    fid = data["candidate"]["fid"]    
    candid =  data["candid"]
    
    # first magnr
    if is_first_detection[fid]:
        first_magnr[fid] = data["candidate"]["magnr"]
        
    # object
    stellar_object, stellar_magstats = validate_object(data["candidate"], is_first_detection[fid])
    corr_detection, corr_magstats, flag = validate_magnitudes(data["candidate"])
    magpsf_corr, sigmapsf_corr, sigmapsf_corr_ref = apply_correction(data["candidate"], first_magnr=first_magnr[fid])
    
    # selected fields for dataframe
    fields = ["fid", "magpsf", "sigmapsf", "isdiffpos", "rfid", "magnr", "distnr", "chinr", "sharpnr", "sgscore1", "distpsnr1", "jd", "ra", "dec", "field", "rcid"]
    df = df.append(pd.DataFrame(data = [[oid, candid] + [magpsf_corr, sigmapsf_corr, sigmapsf_corr_ref] + [data['candidate'][field] for field in fields]  + [stellar_object, stellar_magstats] + [corr_detection, corr_magstats, flag]],
                            columns=[["oid", "candid"] + ["magpsf_corr", "sigmapsf_corr", "sigmapsf_corr_ref"] + fields][0]  + ["stellar_o", "stellar_m"] + ["corr_d", "corr_m", "flag"]))
    
    is_first_detection["object"] = False
    is_first_detection[fid] = False
display(df)

,oid,candid,magpsf_corr,sigmapsf_corr,sigmapsf_corr_ref,fid,magpsf,sigmapsf,isdiffpos,rfid,...,jd,ra,dec,field,rcid,stellar_o,stellar_m,corr_d,corr_m,flag
0,ZTF19aazzpje,899207243115015012,18.081738,0.059556,0.052878,2,19.140087,0.157857,t,632120231,...,2.458654e+06,238.658534,24.690822,632,31,False,False,False,None,False
0,ZTF19aazzpje,899207243115015012,18.081738,0.059556,0.052878,2,19.140087,0.157857,t,632120231,...,2.458654e+06,238.658534,24.690822,632,31,False,False,False,None,False
0,ZTF19aazzpje,899274823115015008,18.495806,0.066319,0.063657,1,19.028122,0.108284,t,632120131,...,2.458654e+06,238.658514,24.690799,632,31,False,False,False,None,False
0,ZTF19aazzpje,903213723115015014,18.294627,0.053420,0.051135,1,18.716547,0.078790,t,632120131,...,2.458658e+06,238.658540,24.690799,632,31,False,False,False,None,False
0,ZTF19aazzpje,903254873115015016,17.980534,0.043276,0.035352,2,18.890068,0.100014,t,632120231,...,2.458658e+06,238.658536,24.690781,632,31,False,False,False,None,False
0,ZTF19aazzpje,906204353115015009,18.345856,0.060400,0.058186,1,18.792978,0.091177,t,632120131,...,2.458661e+06,238.658535,24.690794,632,31,False,False,False,None,False
0,ZTF19aazzpje,906231893115015004,17.938691,0.038494,0.030082,2,18.795702,0.084760,t,632120231,...,2.458661e+06,238.658514,24.690793,632,31,False,False,False,None,False
0,ZTF19aazzpje,909304813115015006,18.037923,0.038213,0.027708,2,19.027611,0.095080,t,632120231,...,2.458664e+06,238.658556,24.690789,632,31,False,False,False,None,False
0,ZTF19aazzpje,912209983115015023,18.541612,0.063033,0.059973,1,19.103943,0.105804,t,632120131,...,2.458667e+06,238.658534,24.690779,632,31,False,False,False,None,False
0,ZTF19aazzpje,912258033115015014,18.093999,0.038523,0.026761,2,19.172894,0.104059,t,632120231,...,2.458667e+06,238.658531,24.690819,632,31,False,False,False,None,False


In [41]:
a = """candid
jd
fid
pid
diffmaglim
isdiffpos
nid
ra
dec
magpsf
sigmapsf
magap
sigmagap
distnr
rb
rbversion
drb
drbversion
magapbig
sigmagapbig
rfid
magpsf_corr
sigmapsf_corr
sigmapsf_corr_ref
has_stamp
corr
flag
candid_alert
step_id_corr"""

In [43]:
print(a.split("\n"))

['candid', 'jd', 'fid', 'pid', 'diffmaglim', 'isdiffpos', 'nid', 'ra', 'dec', 'magpsf', 'sigmapsf', 'magap', 'sigmagap', 'distnr', 'rb', 'rbversion', 'drb', 'drbversion', 'magapbig', 'sigmagapbig', 'rfid', 'magpsf_corr', 'sigmapsf_corr', 'sigmapsf_corr_ref', 'has_stamp', 'corr', 'flag', 'candid_alert', 'step_id_corr']
